In [1]:
from py_ecc.fields import (
    bn128_FQ as FQ,
    bn128_FQ2 as FQ2,
    bn128_FQ12 as FQ12,
)
from py_ecc.bn128 import (
    bn128_curve as curve
)
from field_helper import (
    numberToArray,
    numberToBase,
    hamming_weight,
    printEllipticPoint,
    printFQ,
    printFQ2,
    Fp12convert,
    printFQ12,
    print_fq12_frobenius_coeff
)
import math

In [34]:
p = curve.field_modulus
r = curve.curve_order # baby jubjub prime
x = 4965661367192848881 # parameter x = 0x44e992b44a6909f1
assert p == 36*x**4 + 36*x**3 + 24*x**2 + 6*x + 1
assert r == 36*x**4 + 36*x**3 + 18*x**2 + 6*x + 1
assert p % 4 == 3
assert p % 6 == 1

In [16]:
k = 5
n = math.ceil(254 / k)
print(n)
math.log((18*k)**2 * (2*k-1), 2) + 4*n 

51


220.15363119410165

In [6]:
def print_fq12_frobenius_coeff(n, k):
    gamma = [[0]*6]*12
    for j in range(12):
        gamma[j] = [ FQ2([9,1]) ** ( (i*(p**j-1)//6) % (p**2-1) ) for i in range(6)]
    for j in range(12):
        for i in range(6):
            A, B = gamma[j][i].coeffs
            a = A.n
            b = B.n
            for r in range(k):
                print(f"coeff[{j}][{i}][0][{r}] = {a%(2**n)};")
                a //= 2**n
            assert a == 0
            print("")
            for r in range(k):
                print(f"coeff[{j}][{i}][1][{r}] = {b%(2**n)};")
                b //= 2**n
            assert b == 0
            print("")

def Fp12convert(X, n, k):
    basis1 = X.coeffs
    ret = []
    for i in range(6):
        fq2elt = FQ2([basis1[i].n, 0]) + FQ2([basis1[i+6].n, 0]) * FQ2([9,1])
        ret.append([ numberToArray(fq2elt.coeffs[0].n, n, k) , numberToArray(fq2elt.coeffs[1].n, n, k) ])
    return ret

def printFQ12(X, n, k):
    in62 = Fp12convert(X, n, k)
    print("[")
    for i in range(len(in62)):
        print("[", end="")
        C = in62[i]
        for j in range(len(C)):
            print("[", end="")
            A = C[j]
            for idx in range(len(A)):
                print(f'"{A[idx]}"', end="")
                if idx != len(A)-1:
                    print(",")
            print("]", end="")
            if j != len(C)-1:
                print(",")
        print("]", end="")
        if i != len(in62)-1:
            print(",")
    print("]")

In [5]:
# print_fq12_frobenius_coeff(51,5)

coeff[0][0][0][0] = 1;
coeff[0][0][0][1] = 0;
coeff[0][0][0][2] = 0;
coeff[0][0][0][3] = 0;
coeff[0][0][0][4] = 0;

coeff[0][0][1][0] = 0;
coeff[0][0][1][1] = 0;
coeff[0][0][1][2] = 0;
coeff[0][0][1][3] = 0;
coeff[0][0][1][4] = 0;

coeff[0][1][0][0] = 1;
coeff[0][1][0][1] = 0;
coeff[0][1][0][2] = 0;
coeff[0][1][0][3] = 0;
coeff[0][1][0][4] = 0;

coeff[0][1][1][0] = 0;
coeff[0][1][1][1] = 0;
coeff[0][1][1][2] = 0;
coeff[0][1][1][3] = 0;
coeff[0][1][1][4] = 0;

coeff[0][2][0][0] = 1;
coeff[0][2][0][1] = 0;
coeff[0][2][0][2] = 0;
coeff[0][2][0][3] = 0;
coeff[0][2][0][4] = 0;

coeff[0][2][1][0] = 0;
coeff[0][2][1][1] = 0;
coeff[0][2][1][2] = 0;
coeff[0][2][1][3] = 0;
coeff[0][2][1][4] = 0;

coeff[0][3][0][0] = 1;
coeff[0][3][0][1] = 0;
coeff[0][3][0][2] = 0;
coeff[0][3][0][3] = 0;
coeff[0][3][0][4] = 0;

coeff[0][3][1][0] = 0;
coeff[0][3][1][1] = 0;
coeff[0][3][1][2] = 0;
coeff[0][3][1][3] = 0;
coeff[0][3][1][4] = 0;

coeff[0][4][0][0] = 1;
coeff[0][4][0][1] = 0;
coeff[0][4][0][2] = 0;
coe

In [10]:
import random
rand_twelve = []
for i in range(12):
    rand_twelve.append(random.randrange(p))
X = FQ12(rand_twelve)

In [38]:
Phi12 = p**4 - p**2 + 1
lamb3 = 1
lamb2 = 6*x**2 + 1
lamb1 = -36*x**3 - 18*x**2 - 12*x + 1
lamb0 = -36*x**3 - 30*x**2 - 18*x - 2
Phi12 // r == lamb3 * p**3 + lamb2 * p**2 + lamb1 * p + lamb0 

True

In [43]:
Y = X ** ((p**6 - 1) * (p**2 + 1))

In [56]:
Z = Y ** (Phi12 // r) 

In [75]:
printFQ12(Z, 51, 5)

[
[["1230240385392124",
"357936908694134",
"319960135806541",
"67137439243720",
"281329545235137"],
["1677005579044279",
"105732828588556",
"2020928832000090",
"578203068157553",
"183901047932913"]],
[["230625234712896",
"786959675586198",
"1746080908042125",
"403584908312062",
"813495851716921"],
["1536845135038187",
"1354402396345912",
"376661622282677",
"2071001058307412",
"347839451041051"]],
[["389046412837792",
"467896831404857",
"36908224715553",
"1956642825555426",
"763035301011827"],
["2222725870352709",
"1430217087684263",
"1847503954532002",
"2147810405186655",
"842092461234304"]],
[["1387452211493618",
"928428321931967",
"2038052718346330",
"556014289076437",
"552700864892979"],
["1196299509241438",
"1838563232074719",
"1192057449525785",
"920676941249199",
"94373610719323"]],
[["1193072661480307",
"1397585762870644",
"2230742103912463",
"1318819829618615",
"773527463998786"],
["2066730672277490",
"1206950648341309",
"1132571467697697",
"1377609580293480",
"552765435294739"

In [72]:
x.bit_length()

63